In [5]:
# enum - unique and immutable values , we can perform iteration 
from enum import Enum

class Currency(Enum):
    USD = "USD1"
    EUR = "EUR"

# if we use this currency as a expected structure then we are saying to the model as Only Currency.USD or Currency.EUR are valid values other than this
# It prevents random strings like "Rupees" or "Yen" from being accepted.

#✅ Purpose: to restrict model outputs to a small known set of labels (e.g. currency codes, statuses).

'''
Currency inherits from both str and Enum.

So each member behaves like a string but also has Enum behavior.

Why both?:
If you only wrote Enum, values like "USD" wouldn’t behave like normal strings (you couldn’t use .lower() or serialize easily).
By inheriting from str, Pydantic and JSON treat them as plain strings while still restricting allowed choices.
'''



In [6]:
x =  Currency
x.USD.value

'USD1'

In [7]:
for y in Currency:
    print(y)

Currency.USD
Currency.EUR


In [2]:
# Access environment variables (like your API key)
import os

# Handle JSON encoding/decoding for model outputs.
import json

# Create enumerations (like currencies or payment statuses).
from enum import Enum

# Provide type hints (Optional, List, Literal) for clarity & static checking.
from typing import Optional, List, Literal

# Define structured data models (schema) with validation rules.
from pydantic import BaseModel, Field

# The Google Generative AI SDK, used to interact with Gemini models.
from google import genai



In [4]:
class Currency(str, Enum):
    USD = "USD"
    EUR = "EUR"
    GBP = "GBP"
    CAD = "CAD"
    UNKNOWN = "UNKNOWN"


# When the model extracts text like “USD” or “GBP”, it must map to one of these fixed Enum values.


In [5]:
class PaymentStatus(str, Enum):
    PAID = "PAID"
    PARTIAL = "PARTIAL"
    DUE = "DUE"


In [6]:
# Describes one line item in the invoice table.
# Each row has description, quantity, unit, price, discount, and total.
# Pydantic ensures every key/value matches correct type.

class InvoiceLineItem(BaseModel):
    description: str = Field(description="Raw description of the item or service row.")
    quantity: Optional[float] = Field(default=None, description="Numeric quantity, if found.")
    unit_code: Optional[str] = Field(default=None, description="Unit of measure (e.g., 'hours', 'each', 'lbs').")
    unit_price: Optional[float] = Field(default=None)
    discount_amount: Optional[float] = Field(default=None, description="Discount applied specifically to this line item.")
    line_total_amount: float = Field(description="Total for this line item after calculations.")

# Note : If you don’t inherit BaseModel, Gemini/Pydantic couldn’t generate or validate data for that section.

In [8]:
class SupplierDetails(BaseModel):
    name: str
    legal_address_line1: Optional[str] = Field(default=None)
    # Field is a function from Pydantic
    city: Optional[str] = Field(default=None)
    postcode_zip: Optional[str] = Field(default=None)
    country: Optional[str] = Field(default=None)
    vat_tax_id: Optional[str] = Field(default=None, description="VAT, GST, or Tax identification number.")
    support_email: Optional[str] = Field(default=None)
    support_phone: Optional[str] = Field(default=None)
    website_url: Optional[str] = Field(default=None)

# Represents company issuing the invoice (the seller).
# Many fields are optional — Gemini fills them as null if not found.


"""
vat_tax_id:	defines the attribute name
Optional[str]	means either a string OR None
Field(...)	gives metadata and default behavior
default=None	if no value found, it will store None
description="..."	human-readable explanation; used in generated schemas and documentation
"""

In [9]:
class CustomerDetails(BaseModel):
    company_name: Optional[str] = Field(default=None)
    attention_to_name: Optional[str] = Field(default=None)
    billing_address: Optional[str] = Field(default=None)
    shipping_address: Optional[str] = Field(default=None)
    customer_tax_id: Optional[str] = Field(default=None)

In [10]:
class Invoice(BaseModel):
    # Header Details
    # Basic invoice metadata — IDs, dates, and associated parties.
    invoice_number: str = Field(description="Unique identifier for the invoice.")
    invoice_date: str = Field(description="Date the invoice was issued. ISO 8601 preferred if convertible, else raw string.")
    due_date: Optional[str] = Field(default=None, description="Payment due date.")
    purchase_order_number: Optional[str] = Field(default=None, description="Associated PO number if present.")
    
    # Parties
    supplier: SupplierDetails
    customer: CustomerDetails

    
    
    # Financial breakdown (Optional fields heavily used here for robustness)
    # grand_total_amount is required — everything else can be None.
    currency_code: Currency = Field(default=Currency.USD, description="Detected currency code.")
    subtotal_excluding_tax: Optional[float] = Field(default=None, description="Sum of items before tax.")
    total_tax_amount: Optional[float] = Field(default=None, description="Sum of all tax lines.")
    tax_percentage_rate: Optional[float] = Field(default=None, description="Primary tax rate (e.g., 0.20 for 20%).")
    total_shipping_fees: Optional[float] = Field(default=None)
    total_discount_amount: Optional[float] = Field(default=None, description="Total discount applied to the whole invoice.")
    grand_total_amount: float = Field(description="Final amount to be paid, inclusive of tax/shipping.")
    
    # Payment & Status Meta
    # Captures payment-related info.
    payment_terms_raw: Optional[str] = Field(default=None, description="Raw text terms like 'Net30' or 'Due upon receipt'.")
    payment_status_detected: PaymentStatus = Field(default=PaymentStatus.DUE)
    bank_account_number_detected: Optional[str] = Field(default=None, description="IBAN or account number for payment routing.")

    # Line Items + Quality Flags
    
    # Line items
    items: List[InvoiceLineItem] = Field(default_factory=list, description="Extract all visible line items rows.")

    # Metadata checks
    is_handwritten: bool = Field(default=False, description="Flag True if the document appears to be manually written.")
    has_ocr_errors: bool = Field(default=False, description="Flag True if text has evident scanning noise/garbled text.")


In [13]:
#Step 2: Setup the Gemini Client
import os
from dotenv import load_dotenv

# take environment variables from .env.
load_dotenv() 
api_key = os.getenv("GOOGLE_API_KEY")

client = genai.Client()